In [ ]:
#!/bin/bash
export SUBJECTS_DIR="/data/neuro/MindReader"
#scaffold dset struc
sudo dcm2bids_scaffold -o $home_folder
#dicom in sourcedata path
Dicom_folder=${home_folder}/sourcedata
#transport data from local machine
#local_path="~/Desktop/Dicom/"
#scp -i ~/.ssh/neuro-key.pem -rC $local_path ubuntu@10.0.1.48:/data/neuro/MindReader/sourcedata## instead of using fmriprep
#individual preprocessing steps
declare -a SUBJECT=("sub-01" "sub-02" "sub-03")
#define subjectname file in subject dir
for i in ${SUBJECT[@]};do
    echo ${i}> $SUBJECTS_DIR/${i}/subjectname
done
cd $SUBJECTS_DIR/
#B0 correction
epidewarp.fsl 
#surface-based anatomical preprocessing
for subj in ${SUBJECT}; do
    recon-all -s $subj -all
    #FS to AFNI
    @SUMA_Make_Spec_FS -sid $subj
done
homedir="/data/neuro/Pilot_sep"
indir="$homedir/pipeline"
otdir="$homedir/derivates"

cd $indir
declare -a subjects=("sub-02")

for subj_id in ${subjects[@]};do
      subj_indir=$indir/$subj_id
      subj_otdir=$otdir/$subj_id/run-01
      mkdir -p $subj_otdir
      cd $subj_otdir

      afni_proc.py \
         -blocks tshift align volreg surf blur scale regress \
         -dsets $subj_indir/func/run-01/sub-02_task-listening_run-01_bold.nii \
         -copy_anat $subj_indir/anat/SUMA/brain.nii \
         -surf_anat $subj_indir/anat/SUMA/*_SurfVol.nii \
         -surf_spec $subj_indir/anat/SUMA/[1,2]_?h.spec \
         -tcat_remove_first_trs 10 \
         -volreg_align_to first \
         -volreg_align_e2a \
         -volreg_warp_dxyz 3 \
         -blur_size 6 \
         -regress_stim_times $homedir/behavior/story_1.txt $homedir/behavior/story_2.txt \
         -regress_local_times \
         -test_stim_files no \
         -regress_basis 'GAM' \
         -regress_stim_labels story_1 story_2 \
         -regress_motion_per_run \
         -regress_censor_motion 0.3 \
         -regress_bandpass 0.01 0.1 \
         -regress_opts_3dD \
         -jobs 4 \
         -gltsym 'SYM: story_1' -glt_label 1 story_1 \
         -gltsym 'SYM: story_2' -glt_label 2 story_2 \
         -gltsym 'SYM: story_1 -story_2' -glt_label 3 story1_vs_story2 \
         -gltsym 'SYM: -story_1 +story_1' -glt_label 4 story2_vs_story1 \
         -bash -execute
      cd ../
done
